## Conway's Game of Life

#### Rules
For a space that is 'populated':
* Each cell with one or no neighbors dies, as if by solitude.
* Each cell with four or more neighbors dies, as if by overpopulation.
* Each cell with two or three neighbors survives.
 
For a space that is 'empty' or 'unpopulated'
* Each cell with three neighbors becomes populated.

#### Implementation
We apply above rules in **`n`** dimensions:
* Neighbors defined as all cells within **`k`** distance in any direction from reference cell. Base case **`k=1`**.

In [1]:
# color print
purple = "\x1b[45m"
yellow = "\x1b[43m"
reset = "\x1b[0m"
emph = "\x1b[1m"
neg_color = '\x1b[7m'
purple_text = '\x1b[35m'
neg_purple = neg_color+purple_text

bold = lambda text: emph + str(text) + reset

def highlight(data, color=neg_purple):
    
    return f'{color}{data}{reset}'

print(highlight(bold('  Print Color Templates'+'\t'*8), color='\x1b[7m\x1b[35m'))
for i in range(100):
    
    if i % 5==0:
        print('\n')
        
    print(highlight(f' \\x1b[{i}m ', color=f'\x1b[{i}m'), end=' '*3)
    

  Print Color Templates								


 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

 \x1b     \x1b     \x1b     \x1b     \x1b    

In [23]:
from io import StringIO
import copy
import random

class World:
    
    def __init__(self, *args, space=None, dimensions=None, base_value=0, lookback=10):
        '''
        Initialize world.
        
        TODO:
            - Return subspace (__getitem__ slicing?)
            - Object class for inhabitants
        
        Parameters
        ---
            dimensions: tuple or list of integers
                Dimensions of size of world. Default is (10, 10).
                
            *args: int
                Dimensions of size of world--"dimensions" parameter overrides
                
            base_value:
                Based value to populate space.
        '''
        
        if not dimensions:
            dimensions = args

        if dimensions:
            self.dimensions = dimensions

        else:
            self.dimensions = (10, 10)

        self.dim_size = len(self.dimensions)

        self.space = list(self._generate_space(self.dimensions, base_value))


        self.coordinates = tuple(self._all_coords())
        
        self.lookback = lookback
        self.history = [self._clean_history()]
        self.age = 0
        
    def __repr__(self):
        '''
        Returns state space representation of world.
        '''
        space_str = str(self.space).replace('],','],\n')
        
        for state in self.states:
            if state!=0:
                space_str = space_str.replace(str(state), highlight(state))
        
        buf = StringIO('\n'+space_str)
        
        return buf.getvalue()
        
    def __getitem__(self, coordinates):
        '''
        Returns state of position or subspace from coordinates.
        '''
        subspace = self.space
        
        if type(coordinates)==int:
            return subspace[coordinates]
        
        elif coordinates==None:
            return subspace
        
        for i, pos in enumerate(coordinates):
            if i < self.dim_size:
                subspace=subspace[pos]
                
        return subspace
    
    def __setitem__(self, coordinates, data):
        '''
        Sets state of position or subspace from coordinates.
        '''
        subspace = self.space
        if type(coordinates)==int:
            subspace[coordinates] = data
            
        else:
            for i, pos in enumerate(coordinates):
                if i < self.dim_size - 1:
                    subspace = subspace[pos]
                else:
                    subspace[pos] = data
    
    def __eq__(self, other):
        '''
        If other object type same, returns whether states are equal to the other world.
        Else returns whether self is equal to other object.
        '''
        if type(other) == type(self):
            return self.space == other.space
        
        else:
            return self.space == other
    
    def __add__(self, other):
        '''
        Returns space with added values of intersecting coordinates.
        Else returns space with added values of matching positions in another world.
        '''
        space = self.copy()
        
        for coord in self.coordinates:
            if type(other) == type(self):
                if coord in other:
                    space[coord] = space[coord] + other[coord]
            else:
                space[coord] = space[coord] + other
                
        return space
    
    def __neg__(self):
        '''
        Returns world with negation value in each position.
        '''
        space = self.copy()
        
        for coords in self.coordinates:
            space[coords] = -space[coords]
            
        return space
    
    def __sub__(self, other):
        '''
        Returns world with adding negation of other.
        '''
        return self.__add__(-other)
    
    def add(self, other):
        '''
        Adds other to self and saves history.
        '''
        self = self + other
        return self._save_history()
        
    def sub(self, other):
        '''
        Adds other to self and saves history.
        '''
        self = self - other
        return self._save_history()
    
    def __len__(self):
        '''
        Returns number of dimensions.
        '''
        return self.dim_size
    
    def __contains__(self, item):
        '''
        If item is tuple or list, checks whether item is in coordinates.
        Else checks whether item is in values.
        '''
        if type(item) in (tuple, list):
            return tuple(item) in self.coordinates
        
        else:
            return item in self.states.keys()
        
    
    def _generate_space(self, dimensions, base_value=0, start_dim=0):
        '''
        Generate space based on dimensions recursively.
        
        Called on __init__.
        
        Parameters
        ---
            dimensions: tuple or list of integers
                Dimensions of size of world.
                
            base_value:
                Based value to populate space.
            
        '''
        size = dimensions[start_dim]

        for i in range(size):
            if start_dim >= self.dim_size - 1:
                yield base_value
            else:
                yield list(self._generate_space(dimensions, base_value, start_dim+1))
                
    def steps(self, distance=1, dim_size=None):
        '''
        Returns steps from (0,0) in all directions within distance.
        
        Parameters
        ---
            distance: int
                Maximum number of steps in any direction
            
            dim_size:
                Number of dimensions. Same as dimensions of world if not specified
        '''
        if dim_size==None:
            dim_size=self.dim_size
            
        return tuple(self._recur_steps(distance, dim_size))
    
    def _recur_steps(self, distance, dim_size):
        '''
        Yields steps from (0,0) in all directions within distance recursively. Called in steps()
        
        Parameters
        ---
            distance: int
                Maximum number of steps in any direction
            
            dim_size:
                Number of dimensions. Same as dimensions of world if not specified
        '''
        for i in range(-distance, distance+1):

            if dim_size <= 1:
                yield (i,)
            else:
                for x in self._recur_steps(distance, dim_size-1):
                    yield (i,) + x
    
    
    def _all_coords(self, level=0):
        '''
        Yields position coordinates for each cell in world recursively.
        
        Called on __init__.
        '''
        for i in range(self.dimensions[level]):
            
            if level == self.dim_size-1:
                yield (i, )
                
            elif level < self.dim_size:
                
                for x in self._all_coords(level+1):
                    yield (i,) + x
                    
    def neighbors(self, *args, position=None, blocks=1):
        '''
        Returns sum of state in neighboring cells.
        
        Parameters
        ---
            position: tuple, list
                Coordinate position from which to check neighbors.
            
            blocks: int
                Number of blocks in the neighborhood (largest steps in any direction).
                
            *args: int
                Coordinates from which to check neighbors--position argument overrides.
            
        '''
        
        if position==None:
            position = args
            
        assert len(position)==self.dim_size,             f'Position dimensions {len(position)} do not match {self.dim_size}'
        
        steps = self.steps(blocks, dim_size=len(position))
        
        neighbor_coords = []
        for step in steps:
            
            # neighbor
            coord = tuple(sum(coords) for coords in zip(position, step))
            
            cond = (coord != position) # not itself
            cond = cond and all(x >= 0 for x in coord) # coordinates non-negative
            cond = cond and all(x <= d-1 for x, d in zip(coord, self.dimensions)) # coordinates bounded by dimensions
            
            if cond:
                neighbor_coords.append(coord)
        
        res = 0
        for coord in neighbor_coords:
            res += self.__getitem__(coord)

        return res
    
    @property
    def population(self):
        pop = 0
        for k, v in self.states.items():
            pop += v['Values']
        
        return pop
    
    @property
    def states(self):
        '''
        Returns dictionary of states and sum of states.
        '''
        d = dict()
        for coord in self.coordinates:
            v = self[coord]
            if v in d:
                d[v]['Count'] += 1
                d[v]['Values'] += v
                d[v]['Coordinates'].append(coord)
            else:
                d[v] = {}
                d[v]['Count'] = 1
                d[v]['Values'] = v
                d[v]['Coordinates'] = [coord]

        return d
    
    def sample_coordinates(self, k=10):
        '''
        Returns k random samples of coordinates. 
        '''
        return random.sample(self.coordinates, k=k)

    def seed(self, config, seed_value=1):
        '''
        Seeds coordinates specified in config with value.
        
        Accepts config as list, tuple, or dictionary.
        '''
        
        for coord in config:
            if coord in self.coordinates:
                if type(config) == dict:
                    self[coord] = config[coord]
                else:
                    self[coord] = seed_value
                    
        self = self._save_history()

    def next_generation(self, spawn=3, starve=1, overpop=4):
        '''
        Returns next generation of state space.
        
        Parameters
        ---
            spawn : int, tuple, list
                Number of neighbors needed to spawn (change state from 0 to 1).
            
            starve: int
                Lowerbound for number of neighbors (if neighbours <= starve: change state from 1 to 0).
                
            overpop: int
                Upperboud for number of neighbors (if neighbours >= overpop: change state from 1 to 0).
                
        '''
        space = self.copy()
        
        for cell in self.coordinates:
            
            n = self.neighbors(position=cell)
            
            # already alive
            if self.__getitem__(cell):
                space[cell] = int(starve < n and n < overpop)
            
            else: # not yet living
                if type(spawn)==int:
                    live = (n == spawn)
                else:
                    live = (n in spawn)
                space[cell] = int(live)
        
        return space._save_history()
    
    def replace(self, oldvalue, newvalue, count=None):
        '''
        Replaces values in world.
        '''
        space = self.copy()
        
        for coord in space.coordinates:
            if count==None:
                pass
            elif count<=0:
                break
            else:
                count -= 1
            
            if space[coord] == oldvalue:
                space[coord] = newvalue
        
        return space._save_history()
    
    def copy(self):
        '''
        Returns copy of self.
        '''
        return copy.deepcopy(self)
    
    def _save_history(self):
        '''
        Returns self with current state appended to self.history.
        '''
        self.history.append(self._clean_history())
        self.history = self.history[-self.lookback:]
        
        self.age += 1
        
        return self
    
    def _clean_history(self):
        '''
        Return world with no history to stop history recursion.
        '''
        space = self.copy()
        space.history = None
        return space.space
    
    @property
    def is_stable(self):
        '''
        Returns whether in stable state, either if zero state or current state in lookback history.
        '''
        return (self.population==0) or (self in self.history[:-1])

True

In [20]:
# save interesting seeds for game

def save_config(config, filename='conway_config.txt'):
    
    with open(filename,'a') as f:
        f.write(str(config)+'\n')
    f.close()
    
    print('Configuration Saved')
    print(str(config))


def load_config(filename='conway_config.txt'):

    with open(filename,'r') as f:
        data = f.read()
    f.close()
    data = data.replace('\n',',')
    return eval(f'[{data}]')

In [57]:
from IPython.display import clear_output

def simulate(dimensions=None, seeds=None, k=10, limit=1000, lookback=10, update=0, show=True):
    '''
    Simulate world
    '''
    
    def printif(*args, **kwargs):
        if show:
            print(*args, **kwargs)
    
    if dimensions:
        w = World(*dimensions)
    else:
        w = World()
            
    if seeds==None:
        seeds=w.sample_coordinates(k=k)
    
    # seed values
    w.seed(seeds)
    init_w = w.copy() # initial state
    print(bold('Initial State'))
    print('Population ' + highlight(bold(f' {init_w.population} ')))
    print(init_w)
    print('-'*10)

    # set prev for steady state
    i = 1

    while True:
        
        if update and i % update==0:
            
            if show:
                clear_output()
                print(bold('Initial State'))
                print('Population ' + highlight(bold(f' {init_w.population} ')))
                print(init_w)
                print('-'*10)
                
            print(f'Generation {i} | Population: {w.population}')
            
        w = w.next_generation()
        printif(bold(f'\nGeneration {i}'))
        printif(f'Population:', highlight(f' {w.population} ', color=yellow+emph))
        printif(w)
        
        i += 1

        # check end
        if w.population==0:
            printif('\n'+bold('Died 💀👻'))
            end_state = 'Dead'
            break

        elif w.is_stable:
            printif('\n'+bold('Steady State Achieved 🧬✌'))
            end_state = 'Stable'
            break

        elif i > limit:
            printif('\n'+bold('To infinity and beyond 🙌🚀'))
            end_state = 'Overlimit'
            break
    
    printif('\n'+'='*20)
    printif(f'Survivors: {w.population}')
    printif(f'Age: {i}')
    
    res = {
        'End State'   :  end_state,
        'Seeds'       :  seeds,
        'Kernel'      :  w,
    }
    
    return res

In [59]:
res = simulate()

Initial State
Population  10 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]
----------

Generation 1
Population:  4 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 2
Population:  4 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 

In [60]:
results = []
n=100 # number of simulations
update=0 # update cycle

for i in range(n):
    
    # status update
    if update and i % update==0:
        
        clear_output()
        print('-'*25)
    
    # simulate
    print(f'[{i+1} of {n}] simulations')
    sim_res = simulate()
    results.append(sim_res)

print('='*20)

[1 of 100] simulations
Initial State
Population  10 

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
----------

Generation 1
Population:  2 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 2
Population:  0 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0


Generation 4
Population:  4 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Steady State Achieved 🧬✌

Survivors: 4
Age: 5
[13 of 100] simulations
Initial State
Population  10 

[[0, 0, 1, 0, 0, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]
----------

Generation 1
Population:  2 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],



Generation 4
Population:  6 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 5
Population:  5 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 6
Population:  3 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1,


Generation 5
Population:  6 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Steady State Achieved 🧬✌

Survivors: 6
Age: 6
[33 of 100] simulations
Initial State
Population  10 

[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0]]
----------

Generation 1
Population:  0 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],



Generation 1
Population:  10 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 2
Population:  5 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 3
Population:  2 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0

...


Generation 2
Population:  0 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Died 💀👻

Survivors: 0
Age: 3
[65 of 100] simulations
Initial State
Population  10 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0]]
----------

Generation 1
Population:  0 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 


Generation 2
Population:  7 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 3
Population:  6 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 4
Population:  7 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 1, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0,


Generation 6
Population:  2 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 7
Population:  0 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Died 💀👻

Survivors: 0
Age: 8
[77 of 100] simulations
Initial State
Population  10 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0,


Generation 1
Population:  8 

[[0, 0, 0, 0, 0, 1, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 2
Population:  4 

[[0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Generation 3
Population:  2 

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0,


Generation 1
Population:  0 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

Died 💀👻

Survivors: 0
Age: 2
[94 of 100] simulations
Initial State
Population  10 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 1, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
----------

Generation 1
Population:  5 

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 